# A Quick Introduction to Optuna

This Jupyter notebook goes through the basic usage of Optuna.

- Install Optuna
- Write a training algorithm that involves hyperparameters
  - Read train/valid data
  - Define and train model
  - Evaluate model
- Use Optuna to tune the hyperparameters (hyperparameter optimization, HPO)
- Visualize HPO

## Install `optuna`

Optuna can be installed via `pip` or `conda`.

In [1]:
!pip install --quiet optuna

     |████████████████████████████████| 204kB 3.2MB/s 
     |████████████████████████████████| 1.1MB 16.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 122kB 23.4MB/s 
     |████████████████████████████████| 112kB 26.5MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 


In [2]:
import optuna

optuna.__version__

'1.5.0'

## Optimize Hyperparameters

### Define a simple scikit-learn model

We start with a simple random forest model to classify flowers in the Iris dataset. We define a function called `objective` that encapsulates the whole training process and outputs the accuracy of the model.

In [3]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection

def objective():
    iris = sklearn.datasets.load_iris()  # Prepare the data.
    
    clf = sklearn.ensemble.RandomForestClassifier(    
        n_estimators=5, max_depth=3)  # Define the model.
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()  # Train and evaluate the model.

print('Accuracy: {}'.format(objective()))

Accuracy: 0.9466666666666667


### Optimize hyperparameters of the model

The hyperparameters of the above algorithm are `n_estimators` and `max_depth` for which we can try different values to see if the model accuracy can be improved. The `objective` function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [4]:
import optuna

def objective(trial):
    iris = sklearn.datasets.load_iris()
    
    n_estimators = trial.suggest_int('n_estimators', 2, 20)
    max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))
    
    clf = sklearn.ensemble.RandomForestClassifier(
        n_estimators=n_estimators, max_depth=max_depth)
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2020-07-26 17:57:13,343] Finished trial#0 with value: 0.96 with parameters: {'n_estimators': 15, 'max_depth': 6.194339836265156}. Best is trial#0 with value: 0.96.
[I 2020-07-26 17:57:13,533] Finished trial#1 with value: 0.9533333333333333 with parameters: {'n_estimators': 5, 'max_depth': 3.8058862033007914}. Best is trial#0 with value: 0.96.
[I 2020-07-26 17:57:13,750] Finished trial#2 with value: 0.96 with parameters: {'n_estimators': 15, 'max_depth': 6.30361710928111}. Best is trial#0 with value: 0.96.
[I 2020-07-26 17:57:13,983] Finished trial#3 with value: 0.96 with parameters: {'n_estimators': 19, 'max_depth': 5.1879576620331385}. Best is trial#0 with value: 0.96.
[I 2020-07-26 17:57:14,164] Finished trial#4 with value: 0.96 with parameters: {'n_estimators': 4, 'max_depth': 4.248936004432133}. Best is trial#0 with value: 0.96.
[I 2020-07-26 17:57:14,397] Finished trial#5 with value: 0.9533333333333333 with parameters: {'n_estimators': 17, 'max_depth': 1.672854268732372}. Best 

Accuracy: 0.9733333333333333
Best hyperparameters: {'n_estimators': 9, 'max_depth': 5.554474318535066}


It is possible to condition hyperparameters using Python `if` statements. We can for instance include another classifier, a support vector machine, in our HPO and define hyperparameters specific to the random forest model and the support vector machine.

In [5]:
import sklearn.svm

def objective(trial):
    iris = sklearn.datasets.load_iris()

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 2, 20)
        max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2020-07-26 17:57:46,627] Finished trial#0 with value: 0.32 with parameters: {'classifier': 'SVC', 'svc_c': 1.6086246778500126e-06}. Best is trial#0 with value: 0.32.
[I 2020-07-26 17:57:46,787] Finished trial#1 with value: 0.96 with parameters: {'classifier': 'SVC', 'svc_c': 0.3586156212218282}. Best is trial#1 with value: 0.96.
[I 2020-07-26 17:57:47,010] Finished trial#2 with value: 0.8466666666666667 with parameters: {'classifier': 'RandomForest', 'n_estimators': 14, 'max_depth': 1.9660456944915936}. Best is trial#1 with value: 0.96.
[I 2020-07-26 17:57:47,175] Finished trial#3 with value: 0.96 with parameters: {'classifier': 'SVC', 'svc_c': 59.077355701918}. Best is trial#1 with value: 0.96.
[I 2020-07-26 17:57:47,385] Finished trial#4 with value: 0.9533333333333333 with parameters: {'classifier': 'RandomForest', 'n_estimators': 8, 'max_depth': 2.584188351905201}. Best is trial#1 with value: 0.96.
[I 2020-07-26 17:57:47,555] Finished trial#5 with value: 0.32 with parameters: {'c

Accuracy: 0.9666666666666667
Best hyperparameters: {'classifier': 'RandomForest', 'n_estimators': 14, 'max_depth': 4.70974563063755}


### Plotting the study

Plotting the optimization history of the study.

In [6]:
optuna.visualization.plot_optimization_history(study)

Plotting the accuracies for each hyperparameter for each trial.

In [7]:
optuna.visualization.plot_slice(study)

Plotting the accuracy surface for the hyperparameters involved in the random forest model.

In [8]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])